In [5]:
import pandas as pd
import string
import numpy as np
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

import gensim
from gensim.parsing.preprocessing import STOPWORDS

In [6]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print(train.shape)
print(test.shape)

(200000, 3)


In [7]:
def stop_words():
    return set(stopwords.words('english') + list(STOPWORDS) + list(string.punctuation))

def stem_lemmat(token):
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(token))

def tokenize_word(questions):
    words = []
    for eachQues in questions:
        words.append(word_tokenize(eachQues))
    return words

def process(question):
    stopWords = stop_words()
    processedQues = []
    #processedWord = []
    for eachQues in question:
        no_stpwrds = [word.lower() for word in eachQues if word.lower() not in stopWords]
        #stem_lemmatise = [stem_lemmat(word) for word in no_stpwrds]
        #processedWord.append(stem_lemmatise)
        #processedQues.append(" ".join(stem_lemmatise))
        processedQues.append(" ".join(no_stpwrds))
    return processedQues


def processed_dataset(dataframe, flag=1):
    #remove the nan values
    dataframe['question1'].replace('', np.nan, inplace=True)
    dataframe['question2'].replace('', np.nan, inplace=True)
    dataframe.dropna(subset=['question1', 'question2'], inplace=True)
    
    #word tokenize
    word1 = tokenize_word(dataframe["question1"])
    word2 = tokenize_word(dataframe["question2"])
    
    #processed questions and important tokens
    processed_question1 = process(word1)
    processed_question2 = process(word2)
    
    #creating dataframe
    if flag == 0:
        dataset = pd.DataFrame({'id':dataframe["id"],'qid1':dataframe['qid1'], 'qid2':dataframe['qid2'],\
                                'question1':processed_question1, 'question2':processed_question2, \
                                'is_duplicate': dataframe["is_duplicate"]},\
                           columns=['id','qid1','qid2','question1','question2','is_duplicate'])
        return dataset
    else:
        dataset = pd.DataFrame({'test_id': dataframe["test_id"],'question1':processed_question1,\
                                'question2':processed_question2},\
                           columns=['test_id','question1','question2'])
        return dataset
        

In [4]:
train_df = processed_dataset(train, 0)

In [8]:
test_df = processed_dataset(test)

In [9]:
train_df.to_csv("ptrain.csv", index = False)

In [10]:
test_df.to_csv("ptest.csv", index = False)